# Prueba el modelo reentrenado con 'Transfer Learning con TF-Hub' para reconocer los Tipos de Flores Iris 

0) Instalar el paquete:

In [ ]:
# Luego de instalar el paquete tal vez necesario reiniciar el entorno (ver mensajes que genera)
try:
  # %tensorflow_version only exists in Colab.
  !pip install -q tf-nightly
except Exception:
  pass

!pip install -q -U tf-hub-nightly
!pip install -q tfds-nightly

1) Importar librerías:

In [ ]:
# For running inference on the TF-Hub module.
import tensorflow as tf
import tensorflow_hub as hub

# For downloading the image.
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import layers
from keras.preprocessing import image
from IPython.display import Image

from sklearn.metrics.classification import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import csv

print("\nLibrerías importadas")

2) Montar el Drive:

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# directorio local en Google Drive
path = 'gdrive/My Drive/IA/demo IRIS' 
model_export_path = path + '/Model_TFHub'
imagPath = path + '/IRIS/test'

2) Cargar el modelo ya reentrenado:

In [ ]:
# importa el modelo reentrenado
classifier = tf.keras.models.load_model(model_export_path)
print("\nModelo recuperado de ", model_export_path)

# define el tamaño de imágenes soportadas
with open( model_export_path + '/imagshape.csv', mode='r') as csvfile:
    aux_imShape = list(csv.reader(csvfile))[0]
IMAGE_SHAPE = [ int(aux_imShape[0]), int(aux_imShape[1]) ] 
print('Tamaño de imágenes: ', IMAGE_SHAPE, ' recuperada')

# importa definición de las clases
with open( model_export_path + '/clases.csv', mode='r') as csvfile:
    all_classes = list(csv.reader(csvfile))[0]
print('Definición de las clases: ', all_classes, ' recuperada')

3) Ejecutar el nuevo modelo re-entrenado sobre las imágenes y mostrar estadísticas:

In [ ]:
# define función auxiliar para mostrar resultado de cada imágen 
def testImage(file_name, image_sample, classDesired, showPredictOK):
    result = classifier.predict(image_sample)
    
    # identifica mejor
    bestPos = np.argmax(result, axis=1)
    clasPred = str(all_classes[int(bestPos)]).lower()

    prediction = clasPred + "" + str(result[0][bestPos]) + "" 
    if clasPred == classDesired:
      res = True      
    else:
      res = False
      prediction = prediction + "!"

    # muestra resultados (solo con error)
    if ((not res) or showPredictOK):
      print("> ", file_name,": " , prediction)
      img = Image(file_name, width = "100", height = "80")
      display(img)

    return res, clasPred

# define función auxiliar para mostrar resultado de cada directorio
def testAllClass(classDesired):
  cantOK = 0
  cantNOK = 0
  predict_path =  imagPath + '/' + str(classDesired)
  print("\n")
  y_classReal = []
  y_classRes = []
  for file in os.listdir(predict_path):
      if not file.startswith('.'):
          file = predict_path + "/" + file

          image_sample = image.load_img(file, target_size = IMAGE_SHAPE)
          image_sample = image.img_to_array(image_sample)/255
          image_sample = np.expand_dims(image_sample, axis = 0)
          
          result, clRes = testImage(file, image_sample, classDesired, True)
          if (result):
            cantOK = cantOK + 1
          else:
            cantNOK = cantNOK + 1

          y_classReal.append(classDesired)
          y_classRes.append(clRes)

  print("\nTOTAL CLASS", classDesired,": ", cantOK+cantNOK, ": Detectado OK ", cantOK, "imágenes - Detectado con Error ", cantNOK, "imágenes.")  
  print('con una Exactitud de %f' % accuracy_score(y_classReal, y_classRes))

  return cantOK, cantNOK, y_classReal, y_classRes

# procesa las imágenes de la carpeta <Test>
y_tests = []
y_preds = []
okGral = 0 
NokGral = 0
all_dirs = os.listdir(imagPath)
for each_dir in all_dirs:
  if os.path.isdir(  imagPath + '/' + each_dir):
    print("\n--- Procesando ", each_dir)
    ok, nok, tests, preds = testAllClass(each_dir.lower())
    print("\n--------------------------------------------------------------------------------------------------------------- ")  
    okGral = ok + okGral 
    NokGral = nok + NokGral
    y_tests.extend(tests)
    y_preds.extend(preds)

print("\n===========================================================================================================================")
print("\n= TOTAL GENERAL ", okGral+NokGral, ": Detectado OK ", okGral, "imágenes - Detectado con Error ", NokGral, "imágenes.")

print('\n= Exactitud: %f' % accuracy_score(y_tests, y_preds))
  
print("\n= Matriz de Confusión (real / predecido): ")
cm = confusion_matrix(y_tests, y_preds, labels=[s.lower() for s in all_classes])
cmtx = pd.DataFrame(
    cm, 
    index=['r:{:}'.format(x) for x in all_classes], 
    columns=['p:{:}'.format(x) for x in all_classes]
  )
print(cmtx)

print("\n= Reporte de Clasificación: ")
print(classification_report(y_tests, y_preds))

print("\n===========================================================================================================================")
